In [97]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Подключение библиотек и скриптов 

In [98]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [99]:
import warnings
warnings.filterwarnings('ignore')

## Загрузка данных

In [100]:
# пути к директориям и файлам
TRAIN_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/train.csv'
TEST_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/test.csv'

In [101]:
# считываем датасеты
train_df = pd.read_csv(TRAIN_DATASET_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)

#### Описание датасета

- __Id__ - _идентификационный номер квартиры_
- __DistrictId__ - _идентификационный номер района_
- __Rooms__ - _количество комнат_
- __Square__ - _площадь_
- __LifeSquare__ - _жилая площадь_
- __KitchenSquare__ - _площадь кухни_
- __Floor__ - _этаж_
- __HouseFloor__ - _количество этажей в доме_
- __HouseYear__ - _год постройки дома_
- __Ecology_1, Ecology_2, Ecology_3__ - _экологические показатели местности_
- __Social_1, Social_2, Social_3__ - _социальные показатели местности_
- __Healthcare_1, Helthcare_2__ - _показатели местности, связанные с охраной здоровья_
- __Shops_1, Shops_2__ - _показатели, связанные с наличием магазинов, торговых центров_
- __Price__ - _цена квартиры_


In [102]:
train_df.rename(columns={'Helthcare_2': 'Healthcare_2'}, inplace=True)
test_df.rename(columns={'Helthcare_2': 'Healthcare_2'}, inplace=True)

In [103]:
train_df.sample(5)

In [104]:
test_df.sample(5)

## 1. EDA

In [105]:
plt.style.use('seaborn-deep')

### Целевая переменная

In [106]:
plt.figure(figsize = (16, 8))

plt.subplot(121)
train_df.Price.hist(density=False, bins=20)
plt.ylabel('Count', fontsize=15)
plt.xlabel('Price', fontsize=15)

plt.subplot(122)
sns.kdeplot(train_df.Price, shade=True)
plt.xlabel('Price', fontsize=15)
plt.ylabel('Density', fontsize=15)

plt.suptitle('Distribution of flat prices', fontsize=20)
plt.show()

### Корреляция между ценой квартиры и признаками

In [107]:
correlation = train_df.corrwith(train_df['Price']).sort_values(ascending=False)
correlation.drop('Price', inplace=True)

In [108]:
plt.figure(figsize = (16,8))
plt.bar(correlation.index, correlation)
plt.xticks(rotation='90')

plt.xlabel('Features', fontsize=15)
plt.ylabel('Correlation', fontsize=15)
plt.title('Feature Correlation', fontsize=20)
plt.show()

### Приведение типов

In [109]:
train_df.dtypes

In [110]:
train_df.Id = train_df.Id.astype(str)
test_df.Id = test_df.Id.astype(str)

### Поиск выбросов

In [111]:
num_feat = train_df.select_dtypes(exclude='object')
train_df[list(num_feat.columns)].hist(figsize=(16,16), density=False)
plt.show()

In [112]:
sns.set(rc={'figure.figsize':(16, 8)})
plot = sns.histplot(train_df, x='Rooms', y='Square')
plot.set_xlim(0, 7)
plot.set_ylim(0, 300)

plt.xlabel('Rooms', fontsize=15)
plt.ylabel('Square', fontsize=15)
plt.title('Square', fontsize=20)

plt.show()

## 2. Обработка датасета - пропуски и выбросы

Как показала практика, тщательная обработка выбросов и пропусков никак не влияет на итоговый результат

In [113]:
# разбиваем датасет на тренинговый и валидационный
train_df, valid_df = train_test_split(train_df, test_size=0.25, shuffle=True, random_state=21)

In [114]:
train_df.head()

In [115]:
valid_df.head()

In [116]:
train_df.describe().T

Заменим значения категориальных признаков Ecology_2, Ecology_3, Shops_2 на дамми переменные

In [117]:
obj_feat = train_df.select_dtypes(include='object')
obj_feat.drop(columns=['Id'], inplace=True)
obj_columns = obj_feat.columns
train_df[obj_columns] = train_df[obj_columns].apply(lambda x: pd.factorize(x)[0])

In [118]:
obj_feat_valid = valid_df.select_dtypes(include='object')
obj_feat_valid.drop(columns=['Id'], inplace=True)
obj_columns_valid = obj_feat_valid.columns
valid_df[obj_columns_valid] = valid_df[obj_columns_valid].apply(lambda x: pd.factorize(x)[0])

In [119]:
obj_feat_test = test_df.select_dtypes(include='object')
obj_feat_test.drop(columns=['Id'], inplace=True)
obj_columns_test = obj_feat_test.columns
test_df[obj_columns_test] = test_df[obj_columns_test].apply(lambda x: pd.factorize(x)[0])

In [120]:
mask = train_df.Square < train_df.LifeSquare
mask_valid = valid_df.Square < valid_df.LifeSquare
mask_test = test_df.Square < test_df.LifeSquare
difference = (train_df[mask].LifeSquare/train_df[mask].Square).quantile(q=(0.001, 0.95))
difference  

Если жилая площадь не более 100 метров и превышает общую площадь в разумных пределах, меняем значения Square и LifeSquare местами

In [121]:
mask2 = ((train_df.LifeSquare/train_df.Square).between(difference.iloc[0], difference.iloc[1])) | (train_df[mask].LifeSquare < 100)
train_df.loc[mask2, ['Square', 'LifeSquare']] = train_df.loc[mask2, ['LifeSquare', 'Square']].values

In [122]:
mask2_valid = ((valid_df.LifeSquare/valid_df.Square).between(difference.iloc[0], difference.iloc[1])) | (valid_df[mask_valid].LifeSquare < 100)
valid_df.loc[mask2_valid, ['Square', 'LifeSquare']] = valid_df.loc[mask2_valid, ['LifeSquare', 'Square']].values

In [123]:
mask2_test = ((test_df.LifeSquare/test_df.Square).between(difference.iloc[0], difference.iloc[1])) | (test_df[mask_test].LifeSquare < 100)
test_df.loc[mask2_test, ['Square', 'LifeSquare']] = test_df.loc[mask2_test, ['LifeSquare', 'Square']].values

Создадим словарь по площади квартир в зависимости от количества комнат и обработаем пропуски и выбросы

In [124]:
square_dict = {}

for i in range(1, 6):
    rooms = train_df[train_df['Rooms'] == i]
    square_dict[i] = (rooms['Square'].median(), 
                      rooms['Square'].quantile(q=[0.2, 0.7]), 
                      rooms['Square'].quantile(q=[0.005, 0.988]))

In [125]:
for k, v in square_dict.items():
    min_val = square_dict[k][1].iloc[0]
    max_val = square_dict[k][1].iloc[1]
    train_df.loc[(train_df.Rooms < 1) | (train_df.Rooms > 5) & (min_val <= train_df.Square) & (train_df.Square <= max_val), 'Rooms'] = k
    valid_df.loc[(valid_df.Rooms < 1) | (valid_df.Rooms > 5) & (min_val <= valid_df.Square) & (valid_df.Square <= max_val), 'Rooms'] = k
    test_df.loc[(test_df.Rooms < 1) | (test_df.Rooms > 5) & (min_val <= test_df.Square) & (test_df.Square <= max_val), 'Rooms'] = k

In [126]:
train_df.Rooms.value_counts()

In [127]:
valid_df.Rooms.value_counts()

In [128]:
test_df.Rooms.value_counts()

Ограничим миниальное и максимальное значение площади квартиры 

In [129]:
train_df.loc[train_df.Square <= 15, 'Square'] = 15
valid_df.loc[valid_df.Square <= 15, 'Square'] = 15
test_df.loc[test_df.Square <= 15, 'Square'] = 15

In [130]:
train_df.loc[train_df.Square > 300, 'Square'] = train_df.Square.median()
valid_df.loc[valid_df.Square > 300, 'Square'] = train_df.Square.median()
test_df.loc[test_df.Square > 300, 'Square'] = train_df.Square.median()

In [131]:
train_df.loc[train_df.Square <= 25, 'Rooms'] = 1
train_df.loc[train_df.Square > 205, 'Rooms'] = 5

valid_df.loc[valid_df.Square <= 25, 'Rooms'] = 1
valid_df.loc[valid_df.Square > 205, 'Rooms'] = 5

test_df.loc[test_df.Square <= 25, 'Rooms'] = 1
test_df.loc[test_df.Square > 205, 'Rooms'] = 5

Орбаботаем пропуски и выбросы по жилой площади квартиры и по площади кухни

In [132]:
condition = ~(train_df.LifeSquare/train_df.Square).between(0.4, 0.75)
train_df.loc[condition, 'LifeSquare'] = train_df[condition].Square * 0.6

In [133]:
condition_valid = ~(valid_df.LifeSquare/valid_df.Square).between(0.4, 0.75)
valid_df.loc[condition_valid, 'LifeSquare'] = valid_df[condition_valid].Square * 0.6

In [134]:
condition_test = ~(test_df.LifeSquare/test_df.Square).between(0.4, 0.75)
test_df.loc[condition_test, 'LifeSquare'] = test_df[condition_test].Square * 0.6

In [135]:
condition2 = (train_df.KitchenSquare < 3) | ((train_df.KitchenSquare + train_df.LifeSquare) >= train_df.Square)
train_df.loc[condition2, 'KitchenSquare'] = train_df[condition2].Square * 0.16

In [136]:
condition2_valid = (valid_df.KitchenSquare < 3) | ((valid_df.KitchenSquare + valid_df.LifeSquare) >= valid_df.Square)
valid_df.loc[condition2_valid, 'KitchenSquare'] = valid_df[condition2_valid].Square * 0.16

In [137]:
condition2_test = (test_df.KitchenSquare < 3) | ((test_df.KitchenSquare + test_df.LifeSquare) >= test_df.Square)
test_df.loc[condition2_test, 'KitchenSquare'] = test_df[condition2_test].Square * 0.16

Обработаем значения этажей и годов строительства дома

In [138]:
condition3 = (train_df.HouseYear < 2000) & (train_df.HouseFloor > 32)
train_df.loc[condition3,'HouseFloor'] = train_df[condition3].Floor

In [139]:
condition3_valid = (valid_df.HouseYear < 2000) & (valid_df.HouseFloor > 32)
valid_df.loc[condition3_valid,'HouseFloor'] = valid_df[condition3_valid].Floor

In [140]:
condition3_test = (test_df.HouseYear < 2000) & (test_df.HouseFloor > 32)
test_df.loc[condition3_test,'HouseFloor'] = test_df[condition3_test].Floor

In [141]:
train_df.loc[train_df.HouseYear > 2020, 'HouseYear'] = random.randint(2000, 2020)
valid_df.loc[valid_df.HouseYear > 2020, 'HouseYear'] = random.randint(2000, 2020)
test_df.loc[test_df.HouseYear > 2020, 'HouseYear'] = random.randint(2000, 2020)

In [142]:
train_df.loc[train_df.HouseFloor == 0, 'HouseFloor'] = train_df.loc[train_df.HouseFloor == 0].Floor
valid_df.loc[valid_df.HouseFloor == 0, 'HouseFloor'] = valid_df.loc[valid_df.HouseFloor == 0].Floor
test_df.loc[test_df.HouseFloor == 0, 'HouseFloor'] = test_df.loc[test_df.HouseFloor == 0].Floor

In [143]:
condition4 = train_df.Floor > train_df.HouseFloor
train_df.loc[condition4, ['Floor', 'HouseFloor']] = train_df.loc[condition4, ['HouseFloor', 'Floor']].values

In [144]:
condition4_valid = valid_df.Floor > valid_df.HouseFloor
valid_df.loc[condition4_valid, ['Floor', 'HouseFloor']] = valid_df.loc[condition4_valid, ['HouseFloor', 'Floor']].values

In [145]:
condition4_test = test_df.Floor > test_df.HouseFloor
test_df.loc[condition4_test, ['Floor', 'HouseFloor']] = test_df.loc[condition4_test, ['HouseFloor', 'Floor']].values

In [146]:
train_df.Healthcare_1.fillna(train_df.Healthcare_1.median(), inplace=True)
valid_df.Healthcare_1.fillna(train_df.Healthcare_1.median(), inplace=True)
test_df.Healthcare_1.fillna(train_df.Healthcare_1.median(), inplace=True)

## 3. Добавление новых признаков

Оценим показатели по районам и сгенерируем признак Environment

In [147]:
district_info = train_df[['DistrictId', 'Ecology_1', 
                          'Social_1', 'Social_2', 
                          'Social_3', 'Healthcare_1', 
                          'Healthcare_2', 'Shops_1']].copy()
district_info

In [148]:
district_info.describe().T

In [149]:
train_df['Environment'] = train_df.Social_1/train_df.Social_1.max()\
                        + train_df.Social_2/train_df.Social_2.max()\
                        + train_df.Social_3/train_df.Social_3.max()\
                        + train_df.Healthcare_1/train_df.Healthcare_1.max()\
                        + train_df.Healthcare_2/train_df.Healthcare_2.max()\
                        + train_df.Shops_1/train_df.Shops_1.max()


In [150]:
valid_df['Environment'] = valid_df.Social_1/valid_df.Social_1.max()\
                        + valid_df.Social_2/valid_df.Social_2.max()\
                        + valid_df.Social_3/valid_df.Social_3.max()\
                        + valid_df.Healthcare_1/valid_df.Healthcare_1.max()\
                        + valid_df.Healthcare_2/valid_df.Healthcare_2.max()\
                        + valid_df.Shops_1/valid_df.Shops_1.max()


In [151]:
test_df['Environment'] = test_df.Social_1/test_df.Social_1.max()\
                        + test_df.Social_2/test_df.Social_2.max()\
                        + test_df.Social_3/test_df.Social_3.max()\
                        + test_df.Healthcare_1/test_df.Healthcare_1.max()\
                        + test_df.Healthcare_2/test_df.Healthcare_2.max()\
                        + test_df.Shops_1/test_df.Shops_1.max()


In [152]:
dists = district_info.groupby(['Ecology_1', 'Social_1', 
                               'Social_2', 'Social_3', 
                               'Healthcare_1', 'Healthcare_2', 
                               'Shops_1'])['DistrictId']

district_data = {}

for data,dID in dists:
    district_data[data] = dID.value_counts()


In [153]:
district_dict = {}

districts = district_info.groupby('DistrictId')

for (dID, ddf) in districts:
    
    district_dict[dID] = {}
    district_dict[dID]['Ecology_1'] = ddf['Ecology_1'].value_counts()
    district_dict[dID]['Social_1'] = ddf['Social_1'].value_counts()
    district_dict[dID]['Social_2'] = ddf['Social_2'].value_counts()
    district_dict[dID]['Social_3'] = ddf['Social_3'].value_counts()
    district_dict[dID]['Healthcare_1'] = ddf['Healthcare_1'].value_counts()
    district_dict[dID]['Healthcare_2'] = ddf['Healthcare_2'].value_counts()
    district_dict[dID]['Shops_1'] = ddf['Shops_1'].value_counts()
        

In [154]:
train_df.info()

In [155]:
valid_df.info()

In [156]:
test_df.info()

In [157]:
plt.figure(figsize = (20,15))

sns.set(font_scale=1.4)

correlation_matrix = train_df.corr()
correlation_matrix = np.round(correlation_matrix, 2)

sns.heatmap(correlation_matrix, annot=True, linewidths=1, cmap='coolwarm')

plt.title('Correlation matrix')
plt.show()


## 4. Обучение модели

Как показала практика, добавление новых фичей не сказывается благоприятно на показателях моделей, поэтому ничего лишнего  использовать в модели не будем

In [158]:
X_train = train_df.drop(columns=['Id', 'Price', 'Environment'])
y_train = train_df.Price

X_valid = valid_df.drop(columns=['Id', 'Price', 'Environment'])
y_valid = valid_df.Price

In [159]:
def evaluate_preds(train_true_values, train_pred_values, test_true_values, test_pred_values):
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

In [168]:
model_xgb = XGBRegressor(colsample_bytree=0.5, learning_rate=0.025, max_depth=6, 
                         min_child_weight=0.5, n_estimators=1000, reg_lambda=0.6, 
                         random_state=21)
model_xgb.fit(X_train, y_train)
y_train_pred_xgb = model_xgb.predict(X_train)
y_valid_pred_xgb = model_xgb.predict(X_valid)

In [169]:
evaluate_preds(y_train, y_train_pred_xgb, y_valid, y_valid_pred_xgb)

In [170]:
features = pd.Series(model_xgb.feature_importances_)
features_names = X_train.columns
features.index = features_names

features.sort_values(ascending=False)

## 5. Получение предсказаний для тестового датасета

In [171]:
X_test = test_df.drop(columns=['Id', 'Environment'])

In [172]:
y_test_pred_xgb = model_xgb.predict(X_test)

In [173]:
predicted_prices = pd.read_csv('/kaggle/input/real-estate-price-prediction-moscow/sample_submission.csv')
predicted_prices.head()

In [174]:
predicted_prices['Price'] = y_test_pred_xgb

In [175]:
predicted_prices.head()

In [179]:
predicted_prices.to_csv('./predicted_prices.csv', index=False)

In [180]:
y_test = pd.read_csv('./predicted_prices.csv')

In [181]:
y_test